# Energy Use Data Cleaning

# Introduction
Data from my energy company, ComEd from the past year. 10_22_2022 to 10_22_2023
* USAGE unites are in kWh

## Source
Data from the [My Green Button](https://secure.comed.com/MyAccount/MyBillUsage/pages/secure/GreenButtonConnectDownloadMyData.aspx) webpage on the ComEd website.

# Goals
* become familiar with the columns in the dataset
* remove redundant data
* clean anomalous data

# Column / header info
* **TYPE**: Electric usage
* **DATE**: Day recorded
* **START TIME**: start of recording in Hour:Minutes
* **END TIME**: end of recording in Hour:Minutes
* **USAGE**: Electric usage in kWh
* **UNITS**: Electric usage metric
* **COST**: amount charged for energy usage
* **NOTES**: useless data

In [117]:
import pandas as pd
import numpy as np
energy_df = pd.read_csv(filepath_or_buffer="energy_use_10_22_22_to_10_22_23.csv", header=4)

In [118]:
print(energy_df.columns)
energy_df.head()

Index(['TYPE', 'DATE', 'START TIME', 'END TIME', 'USAGE', 'UNITS', 'COST',
       'NOTES'],
      dtype='object')


,TYPE,DATE,START TIME,END TIME,USAGE,UNITS,COST,NOTES
0,Electric usage,2022-10-22,00:00,00:29,0.11,kWh,$0.01,NaN
1,Electric usage,2022-10-22,00:30,00:59,0.13,kWh,$0.02,NaN
2,Electric usage,2022-10-22,01:00,01:29,0.09,kWh,$0.01,NaN
3,Electric usage,2022-10-22,01:30,01:59,0.20,kWh,$0.02,NaN
4,Electric usage,2022-10-22,02:00,02:29,0.10,kWh,$0.01,NaN


In [119]:
energy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17520 entries, 0 to 17519
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   TYPE        17520 non-null  object 
 1   DATE        17520 non-null  object 
 2   START TIME  17520 non-null  object 
 3   END TIME    17520 non-null  object 
 4   USAGE       17520 non-null  float64
 5   UNITS       17520 non-null  object 
 6   COST        17520 non-null  object 
 7   NOTES       0 non-null      float64
dtypes: float64(2), object(6)
memory usage: 1.1+ MB


In [120]:
# Printing all the unique values of uninteresting columns
print([energy_df['TYPE'].unique(),
energy_df['UNITS'].unique(), 
energy_df['NOTES'].unique()])

[array(['Electric usage'], dtype=object), array(['kWh'], dtype=object), array([nan])]


In [121]:
# dropping columns with 0 variance
energy_df_clean = energy_df.drop(['TYPE', 'UNITS', 'NOTES'], axis='columns')
energy_df_clean.head()

,DATE,START TIME,END TIME,USAGE,COST
0,2022-10-22,00:00,00:29,0.11,$0.01
1,2022-10-22,00:30,00:59,0.13,$0.02
2,2022-10-22,01:00,01:29,0.09,$0.01
3,2022-10-22,01:30,01:59,0.20,$0.02
4,2022-10-22,02:00,02:29,0.10,$0.01


In [122]:
energy_df_clean['START_DT'] = pd.to_datetime(energy_df['START TIME'], infer_datetime_format=True)
energy_df_clean.head()

,DATE,START TIME,END TIME,USAGE,COST,START_DT
0,2022-10-22,00:00,00:29,0.11,$0.01,2023-10-24 00:00:00
1,2022-10-22,00:30,00:59,0.13,$0.02,2023-10-24 00:30:00
2,2022-10-22,01:00,01:29,0.09,$0.01,2023-10-24 01:00:00
3,2022-10-22,01:30,01:59,0.20,$0.02,2023-10-24 01:30:00
4,2022-10-22,02:00,02:29,0.10,$0.01,2023-10-24 02:00:00


In [123]:
# converted the DATE, START, END columns to datetime format
energy_df_clean['DATE_TIME'] = energy_df_clean['DATE'] + ' ' + energy_df_clean['START TIME']
energy_df_clean['DATE_START'] = pd.to_datetime(energy_df_clean['DATE_TIME'], infer_datetime_format=True)

energy_df_clean['DATE_TIME'] = energy_df_clean['DATE'] + ' ' + energy_df_clean['END TIME']
energy_df_clean['DATE_END'] = pd.to_datetime(energy_df_clean['DATE_TIME'], infer_datetime_format=True)
energy_df_clean.head()

,DATE,START TIME,END TIME,USAGE,COST,START_DT,DATE_TIME,DATE_START,DATE_END
0,2022-10-22,00:00,00:29,0.11,$0.01,2023-10-24 00:00:00,2022-10-22 00:29,2022-10-22 00:00:00,2022-10-22 00:29:00
1,2022-10-22,00:30,00:59,0.13,$0.02,2023-10-24 00:30:00,2022-10-22 00:59,2022-10-22 00:30:00,2022-10-22 00:59:00
2,2022-10-22,01:00,01:29,0.09,$0.01,2023-10-24 01:00:00,2022-10-22 01:29,2022-10-22 01:00:00,2022-10-22 01:29:00
3,2022-10-22,01:30,01:59,0.20,$0.02,2023-10-24 01:30:00,2022-10-22 01:59,2022-10-22 01:30:00,2022-10-22 01:59:00
4,2022-10-22,02:00,02:29,0.10,$0.01,2023-10-24 02:00:00,2022-10-22 02:29,2022-10-22 02:00:00,2022-10-22 02:29:00


In [124]:
# created USAGE Duration column from START & END times
energy_df_clean['USAGE_DUR'] = energy_df_clean['DATE_END'] - energy_df_clean['DATE_START']

energy_df_clean.head()

,DATE,START TIME,END TIME,USAGE,COST,START_DT,DATE_TIME,DATE_START,DATE_END,USAGE_DUR
0,2022-10-22,00:00,00:29,0.11,$0.01,2023-10-24 00:00:00,2022-10-22 00:29,2022-10-22 00:00:00,2022-10-22 00:29:00,0 days 00:29:00
1,2022-10-22,00:30,00:59,0.13,$0.02,2023-10-24 00:30:00,2022-10-22 00:59,2022-10-22 00:30:00,2022-10-22 00:59:00,0 days 00:29:00
2,2022-10-22,01:00,01:29,0.09,$0.01,2023-10-24 01:00:00,2022-10-22 01:29,2022-10-22 01:00:00,2022-10-22 01:29:00,0 days 00:29:00
3,2022-10-22,01:30,01:59,0.20,$0.02,2023-10-24 01:30:00,2022-10-22 01:59,2022-10-22 01:30:00,2022-10-22 01:59:00,0 days 00:29:00
4,2022-10-22,02:00,02:29,0.10,$0.01,2023-10-24 02:00:00,2022-10-22 02:29,2022-10-22 02:00:00,2022-10-22 02:29:00,0 days 00:29:00


In [125]:
unique_durs = (energy_df_clean['USAGE_DUR']).unique()
print(unique_durs)

[ 1740000000000 80940000000000]


In [126]:
weird_usage = energy_df_clean[energy_df_clean['USAGE_DUR']==unique_durs[1]]
weird_usage.head()

,DATE,START TIME,END TIME,USAGE,COST,START_DT,DATE_TIME,DATE_START,DATE_END,USAGE_DUR
723,2022-11-06,01:30,23:59,2.03,$0.26,2023-10-24 01:30:00,2022-11-06 23:59,2022-11-06 01:30:00,2022-11-06 23:59:00,0 days 22:29:00


In [127]:
# drop the row with weird start/end times & verify USAGE_DUR is homagenous
wu_start = weird_usage['DATE_START']
energy_df_clean.set_index('DATE_START', inplace=True)
energy_df_clean.drop(wu_start, inplace=True)
print((energy_df_clean['USAGE_DUR']).unique())

[1740000000000]


# Initial Observations
* Observation0
* Observation1

# Ideas for Improvement
* 
* 